# Pseudobulk data with fraction and median non-zero

For use in plotting, we'll compute "pseudobulk" summary statistics for gene expression based on cell metadata grouping.

Note that this approach doesn't use the careful pseudobulk approaches implemented in packages like `scran` - we're instead taking the fraction of cells in which gene expression was detected (`pct_exp`) and the median of expression in those non-zero cells (`median_expression`).

For use in our visualization tools, we'll group by each of our cell type labeling levels (AIFI_L1, _L2, and _L3), as well as by both cell type and the originating, age-related cohort for these cells.

## Load packages

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

from datetime import date
import hisepy
import numpy as np
import os
import pandas as pd
import pickle
import re
import scanpy as sc
import scipy.sparse as scs

In [2]:
if not os.path.exists('output'):
    os.mkdir('output')

In [3]:
out_files = []

## Helper functions

In [4]:
def read_adata_uuid(h5ad_uuid):
    h5ad_path = '/home/jupyter/cache/{u}'.format(u = h5ad_uuid)
    if not os.path.isdir(h5ad_path):
        hise_res = hisepy.cache_files([h5ad_uuid])
    h5ad_filename = os.listdir(h5ad_path)[0]
    h5ad_file = '{p}/{f}'.format(p = h5ad_path, f = h5ad_filename)
    adata = sc.read_h5ad(h5ad_file)
    return adata

In [5]:
def sparse_nz_median(sparse_mat, feat_names):
    # transpose the matrix so values for each gene are together in memory
    sparse_mat = sparse_mat.transpose().tocsr()
    
    # get dimensions for calculations
    n_cells = sparse_mat.shape[1]
    
    # compute the fraction non-zero per gene (row = axis 1)
    pct_exp = sparse_mat.getnnz(axis = 1) / n_cells
    
    # split non-zero values from the matrix directly
    split_values = np.split(sparse_mat.data, sparse_mat.indptr)
    
    # remove first and last entry in split - indptr starts with 0 and ends with the last value
    # so the first and last split entries are not meaningful
    split_values = split_values[1:-1].copy()
    
    # compute medians for every split
    median_expression = [np.median(x) for x in split_values]

    # assemble a DataFrame with results
    res_df = pd.DataFrame({
        'gene': feat_names,
        'pct_exp': pct_exp,
        'median_expression': median_expression
    })
    
    # replace missing values from splits without any values
    # these were not detected so median expression is 0
    res_df['median_expression'] = res_df['median_expression'].fillna(0)
    
    return res_df

## Retrieve atlas data from HISE

In [6]:
h5ad_uuid = '5b3a0cc8-1811-4126-90c7-e9cdd41459fd'

In [7]:
adata = read_adata_uuid(h5ad_uuid)

In [8]:
adata.shape

(1823666, 33538)

In [9]:
obs = adata.obs

### Summarized expression by cell type

For our UMAP viewer, we plot plot expression at each level of cell type resolution

In [10]:
levels = ['AIFI_L1', 'AIFI_L2', 'AIFI_L3']

In [11]:
%%time
level_results = {}

for level in levels:
    print('>>> {l}'.format(l = level))
    
    level_obs = obs.groupby(level)
    
    level_dict = {}
    for cell_type, type_obs in level_obs:
        type_bc = type_obs['barcodes']
        type_adata = adata[adata.obs['barcodes'].isin(type_bc)]
        
        type_res = sparse_nz_median(type_adata.X, type_adata.var_names.tolist())
        
        level_dict[cell_type] = type_res
        
    level_results[level] = level_dict

>>> AIFI_L1
>>> AIFI_L2
>>> AIFI_L3
CPU times: user 7min 2s, sys: 1min 29s, total: 8min 31s
Wall time: 8min 31s


### Save assembled results for each level

In [12]:
level_combined = {}
for level in levels:
    all_level = pd.concat(level_results[level])
    all_level = all_level.reset_index(drop = False, names = [level, 'row_num'])
    all_level = all_level.drop('row_num', axis = 1)
    level_combined[level] = all_level

In [13]:
for level, df in level_combined.items():
    out_csv = 'output/pbmc-ref_{l}_nz-pct-median_pseudobulk_{d}.csv'.format(l = level, d = date.today())
    df.to_csv(out_csv)
    out_files.append(out_csv)
    out_parquet = 'output/pbmc-ref_{l}_nz-pct-median_pseudobulk_{d}.parquet'.format(l = level, d = date.today())
    df.to_parquet(out_parquet)
    out_files.append(out_parquet)

### Restructure for use in visualization apps

In [14]:
gene_results = {}
for level in levels:
    all_res = pd.concat(level_results[level])
    split_res = all_res.groupby('gene')
    split_dict = {}
    for gene, df in split_res:
        df = df.reset_index(drop = False)
        df = df.drop('level_1', axis = 1)
        df = df.rename({'level_0': 'obs_group'}, axis = 1)
        split_dict[gene] = df
    
    gene_results[level] = split_dict

Check that a gene or two look accurate

In [15]:
gene_results['AIFI_L1']['CD79A']

,obs_group,gene,pct_exp,median_expression
0,B cell,CD79A,0.988365,2.726919
1,DC,CD79A,0.079959,0.725910
2,Erythrocyte,CD79A,0.013926,1.230366
3,ILC,CD79A,0.120853,1.526564
4,Monocyte,CD79A,0.017343,0.818575
5,NK cell,CD79A,0.019809,1.260133
6,Platelet,CD79A,0.011135,2.423751
7,Progenitor cell,CD79A,0.150721,0.677979
8,T cell,CD79A,0.034470,1.093967


In [16]:
gene_results['AIFI_L2']['FCN1']

,obs_group,gene,pct_exp,median_expression
0,ASDC,FCN1,0.055556,0.595097
1,CD8aa,FCN1,0.035210,1.298121
2,CD14 monocyte,FCN1,0.996254,3.177890
3,CD16 monocyte,FCN1,0.933907,2.601019
4,CD56bright NK cell,FCN1,0.048847,1.143188
5,CD56dim NK cell,FCN1,0.049148,1.298217
6,DN T cell,FCN1,0.054066,1.043778
7,Effector B cell,FCN1,0.052255,0.973340
8,Erythrocyte,FCN1,0.070955,1.265223
9,ILC,FCN1,0.046209,1.412349


In [17]:
type_pkl = 'output/pbmc-ref_type_nz-pct-median_pseudobulk_{d}.pkl'.format(d = date.today())
with open(type_pkl, 'wb') as out_file:
    pickle.dump(gene_results, out_file)
out_files.append(type_pkl)

### Summarized expression by cell type and cohort

For our Gene Expression Summary viewer, we plot plot expression at each level of cell type resolution partitioned by each cohort in the reference.

In [18]:
%%time
level_results = {}

for level in levels:
    print('>>> {l}'.format(l = level))
    
    level_obs = obs.groupby(['cohort.cohortGuid', level])
    
    level_dict = {}
    for group_tuple, type_obs in level_obs:
        cohort = group_tuple[0]
        cell_type = group_tuple[1]
        c_t = '{c}_{t}'.format(c = cohort, t = cell_type)
        
        type_bc = type_obs['barcodes']
        type_adata = adata[adata.obs['barcodes'].isin(type_bc)]
        
        type_res = sparse_nz_median(type_adata.X, type_adata.var_names.tolist())
        
        type_res['cohort'] = [cohort] * type_res.shape[0]
        level_dict[c_t] = type_res
        
    level_results[level] = level_dict

>>> AIFI_L1
>>> AIFI_L2
>>> AIFI_L3
CPU times: user 10min 21s, sys: 1min 33s, total: 11min 55s
Wall time: 11min 55s


### Save assembled results for each level

In [19]:
level_combined = {}
for level in levels:
    all_level = pd.concat(level_results[level])
    all_level = all_level.reset_index(drop = False, names = ['group', 'row_num'])
    all_level[level] = [re.sub('.+_', '', x) for x in all_level['group']]
    all_level = all_level.drop(['group','row_num'], axis = 1)
    all_level = all_level[['cohort', level, 'gene', 'pct_exp', 'median_expression']]
    level_combined[level] = all_level

In [20]:
for level, df in level_combined.items():
    out_csv = 'output/pbmc-ref_cohort-{l}_nz-pct-median_pseudobulk_{d}.csv'.format(l = level, d = date.today())
    df.to_csv(out_csv)
    out_files.append(out_csv)
    out_parquet = 'output/pbmc-ref_cohort-{l}_nz-pct-median_pseudobulk_{d}.parquet'.format(l = level, d = date.today())
    df.to_parquet(out_parquet)
    out_files.append(out_parquet)

### Restructure for use in visualization apps

In [21]:
gene_results = {}
for level in levels:
    all_res = pd.concat(level_results[level])
    split_res = all_res.groupby('gene')
    
    split_dict = {}
    for gene, df in split_res:
        df = df.reset_index(drop = False)
        df = df.drop('level_1', axis = 1)
        df['obs_group'] = [re.sub('.+_', '', x) for x in df['level_0']]
        df = df.drop('level_0', axis = 1)
        df = df[['obs_group', 'gene', 'pct_exp', 'median_expression', 'cohort']]
        split_dict[gene] = df
    gene_results[level] = split_dict

Check that a gene or two look accurate

In [22]:
gene_results['AIFI_L1']['CD79A']

,obs_group,gene,pct_exp,median_expression,cohort
0,B cell,CD79A,0.990822,2.746202,BR1
1,DC,CD79A,0.087709,0.714458,BR1
2,Erythrocyte,CD79A,0.020000,2.958432,BR1
3,ILC,CD79A,0.119444,1.399105,BR1
4,Monocyte,CD79A,0.018308,0.798966,BR1
5,NK cell,CD79A,0.020623,1.232686,BR1
6,Platelet,CD79A,0.011520,2.419073,BR1
7,Progenitor cell,CD79A,0.157895,0.659168,BR1
8,T cell,CD79A,0.041634,1.068730,BR1
9,B cell,CD79A,0.988087,2.716038,BR2


In [23]:
cohort_type_pkl = 'output/pbmc-ref_cohort-type_nz-pct-median_pseudobulk_{d}.pkl'.format(d = date.today())
with open(cohort_type_pkl, 'wb') as out_file:
    pickle.dump(gene_results, out_file)
out_files.append(cohort_type_pkl)

## Upload Results to HISE

Finally, we'll use `hisepy.upload.upload_files()` to send a copy of our output to HISE to use for downstream analysis steps.

In [24]:
study_space_uuid = '64097865-486d-43b3-8f94-74994e0a72e0'
title = 'PBMC Reference Pseudobulk Frac Medians {d}'.format(d = date.today())

In [25]:
in_files = [h5ad_uuid]

In [26]:
in_files

['5b3a0cc8-1811-4126-90c7-e9cdd41459fd']

In [27]:
out_files

['output/pbmc-ref_AIFI_L1_nz-pct-median_pseudobulk_2024-03-27.csv',
 'output/pbmc-ref_AIFI_L1_nz-pct-median_pseudobulk_2024-03-27.parquet',
 'output/pbmc-ref_AIFI_L2_nz-pct-median_pseudobulk_2024-03-27.csv',
 'output/pbmc-ref_AIFI_L2_nz-pct-median_pseudobulk_2024-03-27.parquet',
 'output/pbmc-ref_AIFI_L3_nz-pct-median_pseudobulk_2024-03-27.csv',
 'output/pbmc-ref_AIFI_L3_nz-pct-median_pseudobulk_2024-03-27.parquet',
 'output/pbmc-ref_type_nz-pct-median_pseudobulk_2024-03-27.pkl',
 'output/pbmc-ref_cohort-AIFI_L1_nz-pct-median_pseudobulk_2024-03-27.csv',
 'output/pbmc-ref_cohort-AIFI_L1_nz-pct-median_pseudobulk_2024-03-27.parquet',
 'output/pbmc-ref_cohort-AIFI_L2_nz-pct-median_pseudobulk_2024-03-27.csv',
 'output/pbmc-ref_cohort-AIFI_L2_nz-pct-median_pseudobulk_2024-03-27.parquet',
 'output/pbmc-ref_cohort-AIFI_L3_nz-pct-median_pseudobulk_2024-03-27.csv',
 'output/pbmc-ref_cohort-AIFI_L3_nz-pct-median_pseudobulk_2024-03-27.parquet',
 'output/pbmc-ref_cohort-type_nz-pct-median_pseudobul

In [28]:
hisepy.upload.upload_files(
    files = out_files,
    study_space_id = study_space_uuid,
    title = title,
    input_file_ids = in_files
)

output/pbmc-ref_AIFI_L1_nz-pct-median_pseudobulk_2024-03-27.csv
output/pbmc-ref_AIFI_L1_nz-pct-median_pseudobulk_2024-03-27.parquet
output/pbmc-ref_AIFI_L2_nz-pct-median_pseudobulk_2024-03-27.csv
output/pbmc-ref_AIFI_L2_nz-pct-median_pseudobulk_2024-03-27.parquet
output/pbmc-ref_AIFI_L3_nz-pct-median_pseudobulk_2024-03-27.csv
output/pbmc-ref_AIFI_L3_nz-pct-median_pseudobulk_2024-03-27.parquet
output/pbmc-ref_type_nz-pct-median_pseudobulk_2024-03-27.pkl
output/pbmc-ref_cohort-AIFI_L1_nz-pct-median_pseudobulk_2024-03-27.csv
output/pbmc-ref_cohort-AIFI_L1_nz-pct-median_pseudobulk_2024-03-27.parquet
output/pbmc-ref_cohort-AIFI_L2_nz-pct-median_pseudobulk_2024-03-27.csv
output/pbmc-ref_cohort-AIFI_L2_nz-pct-median_pseudobulk_2024-03-27.parquet
output/pbmc-ref_cohort-AIFI_L3_nz-pct-median_pseudobulk_2024-03-27.csv
output/pbmc-ref_cohort-AIFI_L3_nz-pct-median_pseudobulk_2024-03-27.parquet
output/pbmc-ref_cohort-type_nz-pct-median_pseudobulk_2024-03-27.pkl
you are trying to upload file_ids... 

(y/n) y


{'trace_id': '78f8b150-b1a4-48af-a87c-648dcd2d7ad5',
 'files': ['output/pbmc-ref_AIFI_L1_nz-pct-median_pseudobulk_2024-03-27.csv',
  'output/pbmc-ref_AIFI_L1_nz-pct-median_pseudobulk_2024-03-27.parquet',
  'output/pbmc-ref_AIFI_L2_nz-pct-median_pseudobulk_2024-03-27.csv',
  'output/pbmc-ref_AIFI_L2_nz-pct-median_pseudobulk_2024-03-27.parquet',
  'output/pbmc-ref_AIFI_L3_nz-pct-median_pseudobulk_2024-03-27.csv',
  'output/pbmc-ref_AIFI_L3_nz-pct-median_pseudobulk_2024-03-27.parquet',
  'output/pbmc-ref_type_nz-pct-median_pseudobulk_2024-03-27.pkl',
  'output/pbmc-ref_cohort-AIFI_L1_nz-pct-median_pseudobulk_2024-03-27.csv',
  'output/pbmc-ref_cohort-AIFI_L1_nz-pct-median_pseudobulk_2024-03-27.parquet',
  'output/pbmc-ref_cohort-AIFI_L2_nz-pct-median_pseudobulk_2024-03-27.csv',
  'output/pbmc-ref_cohort-AIFI_L2_nz-pct-median_pseudobulk_2024-03-27.parquet',
  'output/pbmc-ref_cohort-AIFI_L3_nz-pct-median_pseudobulk_2024-03-27.csv',
  'output/pbmc-ref_cohort-AIFI_L3_nz-pct-median_pseudobulk

In [29]:
import session_info
session_info.show()